In [ ]:
import pandas as pd
import os
import numpy as np
import openai
import gradio as gr
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
literary_score_df= pd.read_csv('Resources/literary_scored_superheroes.csv')
def simulate_battle(score_a, score_b):
    total_score = score_a + score_b
    prob_a_wins = score_a / total_score
    outcome = np.random.rand()  # Generates a random float between 0 and 1
    return 1 if outcome < prob_a_wins else 0  # Returns 1 if A wins, 0 if B wins

def describe_battle(hero1, hero2):
    # Correctly fetch scores for hero1 and hero2 before using them
    try:
        score_a = literary_score_df.loc[literary_score_df['name'].str.lower() == hero1.lower(), 'hero_total_score'].iloc[0]
        score_b = literary_score_df.loc[literary_score_df['name'].str.lower() == hero2.lower(), 'hero_total_score'].iloc[0]
    except IndexError:
        return "One of the heroes not found in the dataset", ""

    # Simulate battle outcome based on scores
    winner = hero1 if simulate_battle(score_a, score_b) == 1 else hero2
    
    # Generate the battle description using GPT (simplified example)
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",  # Adjust based on your OpenAI API plan
            prompt=f"Describe an epic battle between {hero1} and {hero2}, where {winner} wins.",
            temperature=0.7,
            max_tokens=500
        )
        battle_description = response.choices[0].text.strip()
    except Exception as e:
        return "Failed to generate description", str(e)
    
    return winner, battle_description

# Interface setup
iface = gr.Interface(
    fn=describe_battle,
    inputs=[gr.Textbox(label="Hero 1"), gr.Textbox(label="Hero 2")],
    outputs=[gr.Textbox(label="Winner"), gr.Textbox(label="Battle Description")]
)

iface.launch()